In [1]:
# pip install imbalanced-learn
# pip install lightgbm

import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from imblearn.over_sampling import SMOTE

In [2]:
DATASET_DIRECTORY = '../dataset/CICIoT2023/'

In [3]:
# Defina suas colunas de entrada X e a coluna de saída y
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
    'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
    'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
    'ece_flag_number', 'cwr_flag_number', 'ack_count',
    'syn_count', 'fin_count', 'urg_count', 'rst_count',
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
    'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
    'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
    'Radius', 'Covariance', 'Variance', 'Weight',
]
y_column = 'label'

In [4]:
# Obtém a lista de arquivos CSV no diretório
df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
df_sets.sort()

# Divide os conjuntos de treinamento e teste
training_sets = df_sets[:int(len(df_sets) * 0.8)]
test_sets = df_sets[int(len(df_sets) * 0.8):]

In [5]:
# Inicializa o scaler fora do loop
scaler = StandardScaler()

In [6]:
# Inicializa um dicionário para mapear rótulos de classe
dict_2classes = {}
dict_2classes['DDoS-RSTFINFlood'] = 'Attack'
dict_2classes['DDoS-PSHACK_Flood'] = 'Attack'
dict_2classes['DDoS-SYN_Flood'] = 'Attack'
dict_2classes['DDoS-UDP_Flood'] = 'Attack'
dict_2classes['DDoS-TCP_Flood'] = 'Attack'
dict_2classes['DDoS-ICMP_Flood'] = 'Attack'
dict_2classes['DDoS-SynonymousIP_Flood'] = 'Attack'
dict_2classes['DDoS-ACK_Fragmentation'] = 'Attack'
dict_2classes['DDoS-UDP_Fragmentation'] = 'Attack'
dict_2classes['DDoS-ICMP_Fragmentation'] = 'Attack'
dict_2classes['DDoS-SlowLoris'] = 'Attack'
dict_2classes['DDoS-HTTP_Flood'] = 'Attack'

dict_2classes['DoS-UDP_Flood'] = 'Attack'
dict_2classes['DoS-SYN_Flood'] = 'Attack'
dict_2classes['DoS-TCP_Flood'] = 'Attack'
dict_2classes['DoS-HTTP_Flood'] = 'Attack'


dict_2classes['Mirai-greeth_flood'] = 'Attack'
dict_2classes['Mirai-greip_flood'] = 'Attack'
dict_2classes['Mirai-udpplain'] = 'Attack'

dict_2classes['Recon-PingSweep'] = 'Attack'
dict_2classes['Recon-OSScan'] = 'Attack'
dict_2classes['Recon-PortScan'] = 'Attack'
dict_2classes['VulnerabilityScan'] = 'Attack'
dict_2classes['Recon-HostDiscovery'] = 'Attack'

dict_2classes['DNS_Spoofing'] = 'Attack'
dict_2classes['MITM-ArpSpoofing'] = 'Attack'

dict_2classes['BenignTraffic'] = 'Benign'

dict_2classes['BrowserHijacking'] = 'Attack'
dict_2classes['Backdoor_Malware'] = 'Attack'
dict_2classes['XSS'] = 'Attack'
dict_2classes['Uploading_Attack'] = 'Attack'
dict_2classes['SqlInjection'] = 'Attack'
dict_2classes['CommandInjection'] = 'Attack'

dict_2classes['DictionaryBruteForce'] = 'Attack'

In [8]:
# Loop de treinamento
for train_set in tqdm(training_sets):
    df_train = pd.read_csv(os.path.join(DATASET_DIRECTORY, train_set))
    df_train[X_columns] = scaler.fit_transform(df_train[X_columns])
    
    # Mapeie as classes 'Attack' e 'Benign' para 0 e 1
    df_train[y_column] = df_train[y_column].map(dict_2classes)
    
    # Aplica o SMOTE para balancear os dados
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = smote.fit_resample(df_train[X_columns], df_train[y_column])
    
    lgbm_model = LGBMClassifier()  # Use o LGBMClassifier
    lgbm_model.fit(X_resampled, y_resampled)

  0%|          | 0/135 [00:00<?, ?it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 233087, number of negative: 233087
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027195 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7391
[LightGBM] [Info] Number of data points in the train set: 466174, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


  1%|          | 1/135 [00:07<17:43,  7.93s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 213605, number of negative: 213605
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028852 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7186
[LightGBM] [Info] Number of data points in the train set: 427210, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


  1%|▏         | 2/135 [00:14<16:07,  7.27s/it]

...

 99%|█████████▊| 133/135 [17:52<00:17,  8.78s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 234175, number of negative: 234175
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028692 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7365
[LightGBM] [Info] Number of data points in the train set: 468350, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


 99%|█████████▉| 134/135 [17:58<00:08,  8.19s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 237911, number of negative: 237911
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030503 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7339
[LightGBM] [Info] Number of data points in the train set: 475822, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|██████████| 135/135 [18:05<00:00,  8.04s/it]


In [9]:
# Avaliação no conjunto de teste
y_test = []
y_pred_lgbm = []

for test_set in tqdm(test_sets):
    df_test = pd.read_csv(os.path.join(DATASET_DIRECTORY, test_set))
    df_test[X_columns] = scaler.transform(df_test[X_columns])
    
    # Mapeie as classes 'Attack' e 'Benign' para 0 e 1
    df_test[y_column] = df_test[y_column].map(dict_2classes)
    
    y_test += list(df_test[y_column].values)
    
    # Use o modelo LGBM treinado
    y_pred = list(lgbm_model.predict(df_test[X_columns]))
    y_pred_lgbm += y_pred

100%|██████████| 34/34 [01:27<00:00,  2.56s/it]


In [10]:
# Avalie o desempenho do modelo LightGBM
print("##### LightGBM (2 classes) #####")
print('accuracy_score: ', accuracy_score(y_pred_lgbm, y_test))
print('recall_score: ', recall_score(y_pred_lgbm, y_test, average='macro'))
print('precision_score: ', precision_score(y_pred_lgbm, y_test, average='macro'))
print('f1_score: ', f1_score(y_pred_lgbm, y_test, average='macro'))

##### LightGBM (2 classes) #####
accuracy_score:  0.9957330451624496
recall_score:  0.93148338724052
precision_score:  0.9842347441306949
f1_score:  0.9562539169728801


In [11]:
from sklearn.metrics import accuracy_score, classification_report

# Avalie o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred_lgbm)
report = classification_report(y_test, y_pred_lgbm)

print("Acurácia:", accuracy)
print("Relatório de Classificação:")
print(report)

Acurácia: 0.9957330451624496
Relatório de Classificação:
              precision    recall  f1-score   support

      Attack       1.00      1.00      1.00  10096839
      Benign       0.86      0.97      0.91    243322

    accuracy                           1.00  10340161
   macro avg       0.93      0.98      0.96  10340161
weighted avg       1.00      1.00      1.00  10340161



In [ ]:
#Salvando o Modelo Treinado
import joblib
# Salve o modelo treinado em um arquivo
joblib.dump(lgbm_model, 'modelo_lgbm.pkl')

['modelo_lgbm.pkl']